# Clustering of Toronto Neighborhoods
#### Week 3 assignment for 'Applied Data Science Capstone' course
This Jupiter Notebook is the week-3 assignment of the 'Applied Data Science Capstone' course in Coursera.

<B>Thanks for reviewing!</B>


## Table of contents
* [Download and Explore the dataset (**Question 1: Wikipedia dataset**)](#wikipedia)
* [Geo-localization of each Toronto's neighborhood (**Question 2: Foursquare**)](#foursquare)
* [Explore the venues in Toronto's Neighbourhoods](#explore)
* [Analyzing each neighborhood in Toront](#analysing)
* [Cluster Toronto Neighborhoods](#cluster)
* [Examine the Clusters (**Question 3: Clusters**)](#examine)
* [Conclusions](#conclusions)

# 1. Download and Explore the dataset <a name="wikipedia"></a>
#### Toronto Neighborhoods from Wikipedia
First, we import the needed libraries:
* <B>Numpy, Pandas</B>
* <B>Requests</B>: to handle HTML requests
* <B>BeautifulSoup</B>: to help scrape the information from the Wikipedia HTML page, including <B>lxml parser</B>

<br>The only thing (do not forget): lxml parser needs the Kernel to be restarted after installing, to work properly.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # library to handle requests

!pip install bs4   # Install BeautifulSoup 4
from bs4 import BeautifulSoup
!pip install lxml  # HTML parser for compatible with BS4
import lxml
#!pip install html5lib  # HTML parser for compatible with BS4
#import html5lib

!pip install openpyxl  # To export data to excel

import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

print("\nLibraries imported. \nPlease, don't forget to 'restart' the kernell and re-run the cell to avoid problems with lxml")


Libraries imported. 
Please, don't forget to 'restart' the kernell and re-run the cell to avoid problems with lxml


lxml parser seems not to work until you re-start the kernel, so please: re-start the kernel and execute everything again

#### We import the data from Wikipedia using 'BeautifulSoup', which help us understand HTML

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
website_url = requests.get(url).text

soup = BeautifulSoup(website_url,features='lxml')
print(soup.prettify())  # It shows in an understandable way the HTML code for the page

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"d1725c49-8e84-4011-b283-ae10c03212ee","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":954731857,"wgRevisionId":954731857,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toron

The information we are looking for is in an HTML table of the class <i>'wikitable sortable'</i>, so we will look for that kind of object into the 'soup'. And luckily, it is the first table of that kind that we find.

In [3]:
My_table = soup.find('table',{'class':'wikitable sortable'})  # Select first objetc of that kind
#My_table = soup.find_all('table')[0] # Grab the first table
My_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postal code
</th>
<th>Borough
</th>
<th>Neighborhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park / Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor / Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park / Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern / Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3B
</td>
<td>North York
</td>
<td>Don Mills
</td></tr>
<tr>
<td>M4B
</td>
<td>Ea

#### Now we can populate a DataFrame with the information of the wikipedia table:

In [4]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

# Populate the dataFrame with the Toronto neighbourhoods
for row in My_table.find_all('tr'):     # <tr> is the line id, in an HTML table
    columns = row.find_all('td')        # <td> is the Column id, inside a row of an HTML table
    
    # For each line in the table (except for the title row)
    if (len(row.find_all('td')) != 0):  # skip title row: which doesn't have <td> tags, just <th> ones
        
        # Select the data (removing spaces and \n at the end of the data)
        postal_code = columns[0].get_text().rstrip()
        borough = columns[1].get_text().rstrip()
        neighborhood = columns[2].get_text().rstrip()
        #print(postal_code, borough, neighborhood)
        
        # If neighborhood is empty, the neighborhood is the borough
        if (len(neighborhood) == 0): 
            neighborhood = borough
        
        # I Check if the postal code already exists in the DataFrame: then I append the value os the new neighborhood
        if neighborhoods[neighborhoods['PostalCode'] == postal_code].shape[0] > 0:
            neighborhood = neighborhood + ", " + neighborhoods.loc[neighborhoods.loc[:, 'PostalCode'] == postal_code, 'Neighborhood']
            neighborhoods.loc[neighborhoods.loc[:, 'PostalCode'] == postal_code, 'Neighborhood'] =  neighborhood
            print('repeated postal code: {}'.format(postal_code))
            
        # If borough 'not assigned' then skip it!
        elif (borough.upper() != 'NOT ASSIGNED'):
            neighborhoods = neighborhoods.append({'PostalCode': postal_code,
                                              'Borough': borough,
                                              'Neighborhood': neighborhood
                                             }, ignore_index=True)

neighborhoods.head(12)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,Malvern / Rouge
7,M3B,North York,Don Mills
8,M4B,East York,Parkview Hill / Woodbine Gardens
9,M5B,Downtown Toronto,Garden District / Ryerson


The wikipedia page has change: there is no longer more than one line for each postal code. We check in the following lines that there is no repeated Postal Code in the table above. So, nothing to do with that specific advice in the assignment explanation.
I also take chance to write the data to an excel file.

In [5]:
# I extract the data to excel to be sure that I have what I was looking for
# and to explore and better understand the data
type(neighborhoods)
neighborhoods.to_excel("toronto_neighborhoods.xlsx")

# And finally, I chech that there is no postal code repeated!
different_postal_codes = len(neighborhoods ['PostalCode'].unique())
total_lines = neighborhoods ['PostalCode'].count()

if different_postal_codes == total_lines :
    print('No postal code repeated:')
else:
    print('Attention! Repeated postal codes:')
          
print('{} different postal codes in {} DataFrame lines'.format(different_postal_codes, total_lines))

No postal code repeated:
103 different postal codes in 103 DataFrame lines


<B>Important</B>: There are some neighbourhoods (5 ones: Don Mills, for example) that extend through more than one postal code... As we are really clustering postal codes, we have to work with 'PostalCode+Nighborhood' keys so that our analysis is correct. Otherwise we will be assigning the analysis of a postal code to all the ones that share neighborhood. We will add that key-column in the next cells, not now.

#### Last thing of this chapter: the shape of the selected DataFrame

In [6]:
neighborhoods.shape

(103, 3)

# 2. Geo-localization of each Toronto's neighborhood <a name="foursquare"></a>

#### First thing, again, importing additional libraries...

In [7]:
!pip install geocoder
import geocoder # import geocoder
from io import StringIO

print("\nLibraries imported.")

     |████████████████████████████████| 102kB 5.9MB/s ta 0:00:011
     |████████████████████████████████| 92kB 5.4MB/s  eta 0:00:01
     |████████████████████████████████| 829kB 5.1MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future

Libraries imported.


We define a function that returns the latitude/longitude of a given <B>Postal Code</B> in Toronto, Ontario, to help us find each location.

In [8]:
# Returns latitude and longitude of a given postal code in Toronto, Ontario
def get_coordinates(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    latitude = None 
    longitude = None
    counter = 0
    
    # loop until you get the coordinates
    while(lat_lng_coords is None and counter < 5):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        counter = counter + 1
        print(counter)
    
    if lat_lng_coords is not None:
        latitude = lat_lng_coords[0]
        longitude = lat_lng_coords[1]
    
    return latitude, longitude

#### We take the geo-localization info from the csv file
Finally, geocoder doesn't work: I have seen comments in the forum explaining it doesn't respond, so I'll try with the .csv file...

I get the geo-localization file:

In [9]:
!wget -O geo-data.csv http://cocl.us/Geospatial_data

print('File downloaded!')

--2020-05-04 09:46:26--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 158.85.108.83, 169.48.113.194, 158.85.108.86
Connecting to cocl.us (cocl.us)|158.85.108.83|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2020-05-04 09:46:26--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|158.85.108.83|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-05-04 09:46:28--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.26.197
Connecting to ibm.box.com (ibm.box.com)|107.152.26.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-05-04 09:46:28--  https://ibm.box.com/public

Turn the *.csv file to a DataFrame.

In [10]:
geo_data = pd.read_csv("geo-data.csv")
#geo_data.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Now, it is time to merge both datasets
We also drop the repeated 'Postal Code' column

In [11]:
neighborhoods = pd.merge(neighborhoods, geo_data, left_on='PostalCode', right_on='Postal Code')
neighborhoods.drop('Postal Code', axis=1, inplace=True)
neighborhoods.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,Parkview Hill / Woodbine Gardens,43.706397,-79.309937
9,M5B,Downtown Toronto,Garden District / Ryerson,43.657162,-79.378937


# 3. Explore the venues in Toronto's Neighbourhoods <a name="explore"></a>

#### Define Foursquare credentials and version
Below are the credentials for FourSquare (hopefully hidden)

In [12]:
# @hidden_cell
CLIENT_ID = 'ILFGWMEPJLUOTF4P01CJPWEJ4HTCQQU51MRGVI4B0BSEELEZ' # your Foursquare ID
CLIENT_SECRET = 'yAY0CTHXLR5TR2EATVBX1S3PAHOYL1SHN5WA1XPFUNHTT1BZQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT=100
ACCESS_TOKEN = '5KMXQWXZI3RGOI5S3LAAZWYF1QL5UTBAYQ1GVQUE3O10W1RV'    # Authentication Token: see instructions in Coursera

# print('Your credentails:')
# print('CLIENT_ID: ' + CLIENT_ID)
# print('CLIENT_SECRET:' + CLIENT_SECRET)

#### Let's use the function defined in 'New York clustering' to get the venues information from all the neighborhoods in Toronto
The function is based on the coordinates to look for the venues, the 'Neighborhood' is only a reference to recover the info from the dataset (we will use 'Postal_Code'+'Neighborhood' to avoide missuse of information.

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    count = 1
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(count, name)
        count = count + 1
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&oauth_token={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            ACCESS_TOKEN
        )
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### We create a new DataFrame with Toronto's venues by neighborhood

First, we include a new column 'Postal_Code' + 'Neighborhood'

In [14]:
neighborhoods['Extended_Neighborhood'] = neighborhoods['PostalCode']+": "+neighborhoods['Neighborhood']
neighborhoods.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Extended_Neighborhood
0,M3A,North York,Parkwoods,43.753259,-79.329656,M3A: Parkwoods
1,M4A,North York,Victoria Village,43.725882,-79.315572,M4A: Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,M5A: Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763,M6A: Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494,M7A: Queen's Park / Ontario Provincial Government


Now, we go for the venues info:<br>
(I get 'toronto_vanues_byPostalCode' for later analysis of venues by Postal Code, to help me understand the clusters)

In [15]:
toronto_venues = getNearbyVenues(names=neighborhoods['Extended_Neighborhood'],
                                 latitudes=neighborhoods['Latitude'],
                                 longitudes=neighborhoods['Longitude']
                                )
toronto_venues_byPostalCode = getNearbyVenues(names=neighborhoods['PostalCode'],
                                 latitudes=neighborhoods['Latitude'],
                                 longitudes=neighborhoods['Longitude']
                                )

1 M3A: Parkwoods
2 M4A: Victoria Village
3 M5A: Regent Park / Harbourfront
4 M6A: Lawrence Manor / Lawrence Heights
5 M7A: Queen's Park / Ontario Provincial Government
6 M9A: Islington Avenue
7 M1B: Malvern / Rouge
8 M3B: Don Mills
9 M4B: Parkview Hill / Woodbine Gardens
10 M5B: Garden District / Ryerson
11 M6B: Glencairn
12 M9B: West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale
13 M1C: Rouge Hill / Port Union / Highland Creek
14 M3C: Don Mills
15 M4C: Woodbine Heights
16 M5C: St. James Town
17 M6C: Humewood-Cedarvale
18 M9C: Eringate / Bloordale Gardens / Old Burnhamthorpe / Markland Wood
19 M1E: Guildwood / Morningside / West Hill
20 M4E: The Beaches
21 M5E: Berczy Park
22 M6E: Caledonia-Fairbanks
23 M1G: Woburn
24 M4G: Leaside
25 M5G: Central Bay Street
26 M6G: Christie
27 M1H: Cedarbrae
28 M2H: Hillcrest Village
29 M3H: Bathurst Manor / Wilson Heights / Downsview North
30 M4H: Thorncliffe Park
31 M5H: Richmond / Adelaide / King
32 M6H: Dufferin / Dovercourt

#### ATTENTION: Now, we save the venues to a 'csv' file to avoid over-using the daily FourSquare service

In [16]:
# Execute always after reading from FourSquare, to save the data
toronto_venues_byPostalCode.to_csv('toronto_venues_byPostalCode.csv')
toronto_venues.to_csv('toronto_venues.csv')

#### ATTENTION: We can reload the venues from the csv file instead of asking again to FourSquare if we need to refresh the DataFrame
(Not necessary to be executed for presentation)

In [ ]:
# Execute always you need to reload FourSquare data
toronto_venues_byPostalCode = pd.read_csv('toronto_venues_byPostalCode.csv')
toronto_venues = pd.read_csv('toronto_venues.csv')

Let's see the shape and first 5 rows of toronto_venues

In [17]:
print(toronto_venues.shape)
toronto_venues.head()

(2716, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A: Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,M3A: Parkwoods,43.753259,-79.329656,Careful & Reliable Painting,43.752622,-79.331957,Construction & Landscaping
2,M3A: Parkwoods,43.753259,-79.329656,649 Variety,43.754513,-79.331942,Convenience Store
3,M3A: Parkwoods,43.753259,-79.329656,Sun Life,43.754760,-79.332783,Construction & Landscaping
4,M3A: Parkwoods,43.753259,-79.329656,GTA Restoration,43.753396,-79.333477,Fireworks Store


Let's check how many venues were returned for each neighborhood

In [18]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
M1B: Malvern / Rouge,2,2,2,2,2,2
M1C: Rouge Hill / Port Union / Highland Creek,5,5,5,5,5,5
M1E: Guildwood / Morningside / West Hill,11,11,11,11,11,11
M1G: Woburn,9,9,9,9,9,9
M1H: Cedarbrae,15,15,15,15,15,15
M1J: Scarborough Village,1,1,1,1,1,1
M1K: Kennedy Park / Ionview / East Birchmount Park,12,12,12,12,12,12
M1L: Golden Mile / Clairlea / Oakridge,11,11,11,11,11,11
M1M: Cliffside / Cliffcrest / Scarborough Village West,6,6,6,6,6,6


#### Let's find out how many unique categories can be curated from all the returned venues

In [19]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 305 uniques categories.


# 4. Analyzing each neighborhood in Toronto <a name="analysing"></a>

#### One hot coding of the venues by neighbourhood

In [20]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns)
fixed_columns.insert(0, fixed_columns.pop(fixed_columns.index('Neighborhood')))
#fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Bath House,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Paper / Office Supplies Store,Park,Peking Duck Restaurant,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Rental Service,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Social Club,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Taco Pl

In [21]:
toronto_onehot.shape

(2716, 305)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [22]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Bath House,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Paper / Office Supplies Store,Park,Peking Duck Restaurant,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Rental Service,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Social Club,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Taco Pl

#### Let's confirm the shape

In [23]:
toronto_grouped.shape

(102, 305)

<B>Knowledge acquired:</B>
* There are 5 neighborhoods that extend themselves across more than one 'PostalCode': we have solved it working with the concatenation of 'Postal Code' + 'Neighborhood' as the key to organize the info from the venues
* Anyway: there is 1 postal code for which FourSquare didn't return any venue: we will have to deal with it later. For now, we will exclude it from the clustering exercise, as we don't have info for it
* There are 305 kinds of venues

#### Let's create a pandas DataFrame with each neighborhood and the top 10 venues by neighborhood

First, let's write a function to sort the venues in descending order.

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [25]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B: Malvern / Rouge,Home Service,Fast Food Restaurant,Yoga Studio,Event Space,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant
1,M1C: Rouge Hill / Port Union / Highland Creek,Construction & Landscaping,Home Service,Golf Course,Moving Target,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
2,M1E: Guildwood / Morningside / West Hill,Electronics Store,Rental Car Location,Intersection,Medical Center,Breakfast Spot,Moving Target,Spa,Bank,Mexican Restaurant,Ethiopian Restaurant
3,M1G: Woburn,Coffee Shop,Mexican Restaurant,Indian Restaurant,Convenience Store,Pharmacy,Soccer Field,Korean Restaurant,Insurance Office,Dumpling Restaurant,Eastern European Restaurant
4,M1H: Cedarbrae,Caribbean Restaurant,Thai Restaurant,Shipping Store,Medical Center,Burger Joint,Spa,Fried Chicken Joint,Bank,Bakery,Athletics & Sports


That dataframe considers the 97 neighborhoods we have info from

In [26]:
neighborhoods_venues_sorted.shape

(102, 11)

# 5. Cluster Toronto Neighborhoods <a name="cluster"></a>

First, we have to import the mathematical libraries:

In [27]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!pip install webcolors   # To help understand in the map which color is which cluster...
import webcolors

print('Libraries imported.')

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    brotlipy-0.7.0             |py36h8c4c3a4_1000         346 KB  conda-forge
    chardet-3.0.4              |py36h9f0ad1d_1006         188 KB  conda-forge
    cryptography-2.9.2         |   py36h45558ae_0         613 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    pandas-1.0.3               |   py36h830a2c2_1        11.1 MB  conda-

We will run *k*-means to cluster the neighborhood into 10 clusters.
* The model runs over 'toronto_grouped' (without the columns 'Neighborhood'): it uses the values from 0 to 1 to do the clustering
* Later we come back to 'neighborhoods_venue_sorted' to explain each cluster

In [28]:
# set number of clusters
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([5, 3, 2, 0, 2, 6, 0, 2, 2, 2], dtype=int32)

Now, let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [29]:
# Change the name of the column to do the merge with the correct one
neighborhoods_venues_sorted.rename(columns = {'Neighborhood':'Extended_Neighborhood'}, inplace=True)

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

#toronto_merged = toronto_data
toronto_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Extended_Neighborhood'), on='Extended_Neighborhood')
toronto_merged = toronto_merged.drop('Extended_Neighborhood', axis=1)

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3.0,Construction & Landscaping,BBQ Joint,Food & Drink Shop,Bus Stop,Fireworks Store,Convenience Store,Park,Coworking Space,Fabric Shop,Eastern European Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Financial or Legal Service,Hockey Arena,Pizza Place,Coffee Shop,Intersection,French Restaurant,Portuguese Restaurant,Ethiopian Restaurant,Drugstore,Dumpling Restaurant
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,2.0,Coffee Shop,Café,Pub,Park,Bakery,Breakfast Spot,Theater,Yoga Studio,Cosmetics Shop,Shoe Store
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763,2.0,Clothing Store,Furniture / Home Store,Accessories Store,Women's Store,Home Service,Carpet Store,Athletics & Sports,Event Space,Arts & Crafts Store,Lighting Store
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494,0.0,Coffee Shop,Sushi Restaurant,Diner,Japanese Restaurant,Burger Joint,Spa,Fried Chicken Joint,Distribution Center,Italian Restaurant,Middle Eastern Restaurant


Last thing: we have a PostalCode (M1X: Upper Rouge) without a cluster. So, I will assignt it an 'eleventh color' to draw it in the map but apart from the other clusters

In [30]:
toronto_merged.loc[toronto_merged.loc[:, 'PostalCode'] == "M1X"]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
95,M1X,Scarborough,Upper Rouge,43.836125,-79.205636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
# I assign cluster 10 to M1X Postal Code
# That postal code didn't find venues information in FourSquares, so it stayed out of the clustering exercise
toronto_merged.loc[toronto_merged.loc[:, 'PostalCode'] == "M1X", 'Cluster Labels'] = kclusters

toronto_merged.loc[toronto_merged.loc[:, 'PostalCode'] == "M1X"]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
95,M1X,Scarborough,Upper Rouge,43.836125,-79.205636,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
toronto_merged.shape

(103, 16)

#### Let's find latitude and longitude for Toronto:

In [33]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Finally, let's visualize the resulting clusters

In [34]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set personalized color scheme for the clusters
color_names = ('blue', 'green', 'orange', 'red', 'white', 'black', 'grey', 'pink', 'cyan', 'brown', 'magenta')
rainbow = [webcolors.name_to_hex(name) for name in color_names]

# set color scheme for the clusters
#number_colors = kclusters + 1   # For the missed postal code
#x = np.arange(number_colors)
#ys = [i + x + (i*x)**2 for i in range(number_colors)]
#colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
#rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        #color = 'blue',
        color=rainbow[int(cluster-1)],
        fill=True,
        #fill_color='#3186cc',
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
    
map_clusters

# 6. Examine the Clusters <a name="examine"></a>

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, we can then assign a name to each cluster.

#### Cluster 0: food of the world, with lots of varied restaurants

In [35]:
cluster_number = 0
cluster = toronto_merged.loc[toronto_merged['Cluster Labels'] == cluster_number, toronto_merged.columns[[1, 2] + list(range(5, toronto_merged.shape[1]))]]
print('cluster has {} postal codes included with color {}'.format(cluster.shape[0], webcolors.hex_to_name(rainbow[cluster_number - 1])))
cluster

cluster has 10 postal codes included with color magenta


,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,Victoria Village,0.0,Financial or Legal Service,Hockey Arena,Pizza Place,Coffee Shop,Intersection,French Restaurant,Portuguese Restaurant,Ethiopian Restaurant,Drugstore,Dumpling Restaurant
4,Downtown Toronto,Queen's Park / Ontario Provincial Government,0.0,Coffee Shop,Sushi Restaurant,Diner,Japanese Restaurant,Burger Joint,Spa,Fried Chicken Joint,Distribution Center,Italian Restaurant,Middle Eastern Restaurant
19,East Toronto,The Beaches,0.0,Health Food Store,Trail,Pizza Place,Coffee Shop,Pub,Asian Restaurant,Park,Home Service,Gym / Fitness Center,Electronics Store
22,Scarborough,Woburn,0.0,Coffee Shop,Mexican Restaurant,Indian Restaurant,Convenience Store,Pharmacy,Soccer Field,Korean Restaurant,Insurance Office,Dumpling Restaurant,Eastern European Restaurant
38,Scarborough,Kennedy Park / Ionview / East Birchmount Park,0.0,Pharmacy,Discount Store,Hobby Shop,Department Store,Convenience Store,Comfort Food Restaurant,Bus Station,Coffee Shop,Train Station,Chinese Restaurant
56,York,Del Ray / Mount Dennis / Keelsdale and Silvert...,0.0,Pharmacy,Coffee Shop,Discount Store,Sandwich Place,Bar,Restaurant,Convenience Store,Farmers Market,Farm,Falafel Restaurant
70,Etobicoke,Westmount,0.0,Pizza Place,Medical Supply Store,Discount Store,Intersection,Coffee Shop,Middle Eastern Restaurant,Sandwich Place,Chinese Restaurant,Yoga Studio,Empanada Restaurant
72,North York,Willowdale,0.0,Spa,Butcher,Bank,Pharmacy,Home Service,Discount Store,Pizza Place,Grocery Store,Coffee Shop,Ethiopian Restaurant
76,Mississauga,Canada Post Gateway Processing Centre,0.0,Hotel,Coffee Shop,American Restaurant,Mediterranean Restaurant,Fried Chicken Joint,Gym,Intersection,Middle Eastern Restaurant,Burrito Place,Sandwich Place
93,Etobicoke,Alderwood / Long Branch,0.0,Pizza Place,Skating Rink,Coffee Shop,Sandwich Place,Pub,Dance Studio,Pool,Athletics & Sports,Gym,Pharmacy


#### Cluster 1: From home to the pizza place and back home
Just a 'Pizza Place' venue at hand

In [36]:
cluster_number = 1
cluster = toronto_merged.loc[toronto_merged['Cluster Labels'] == cluster_number, toronto_merged.columns[[1, 2] + list(range(5, toronto_merged.shape[1]))]]
print('cluster has {} postal codes included with color {}'.format(cluster.shape[0], webcolors.hex_to_name(rainbow[cluster_number - 1])))
cluster

cluster has 1 postal codes included with color blue


,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Etobicoke,Islington Avenue,1.0,Pizza Place,Yoga Studio,Fabric Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space


#### Cluster 2: Coffe shops, spas and lots of venues everywhere

In [37]:
cluster_number = 2
cluster = toronto_merged.loc[toronto_merged['Cluster Labels'] == cluster_number, toronto_merged.columns[[1, 2] + list(range(5, toronto_merged.shape[1]))]]
print('cluster has {} postal codes included with color {}'.format(cluster.shape[0], webcolors.hex_to_name(rainbow[cluster_number - 1])))
cluster

cluster has 74 postal codes included with color green


,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,Regent Park / Harbourfront,2.0,Coffee Shop,Café,Pub,Park,Bakery,Breakfast Spot,Theater,Yoga Studio,Cosmetics Shop,Shoe Store
3,North York,Lawrence Manor / Lawrence Heights,2.0,Clothing Store,Furniture / Home Store,Accessories Store,Women's Store,Home Service,Carpet Store,Athletics & Sports,Event Space,Arts & Crafts Store,Lighting Store
7,North York,Don Mills,2.0,Pharmacy,Japanese Restaurant,Caribbean Restaurant,Café,Restaurant,Baseball Field,Athletics & Sports,Construction & Landscaping,Gym / Fitness Center,Grocery Store
8,East York,Parkview Hill / Woodbine Gardens,2.0,Pizza Place,Pet Store,Intersection,Breakfast Spot,Spa,Bank,Café,Furniture / Home Store,Athletics & Sports,Gastropub
9,Downtown Toronto,Garden District / Ryerson,2.0,Coffee Shop,Clothing Store,Café,Bubble Tea Shop,Middle Eastern Restaurant,Italian Restaurant,Japanese Restaurant,Cosmetics Shop,Burger Joint,Thai Restaurant
10,North York,Glencairn,2.0,Pizza Place,Spa,Metro Station,Sushi Restaurant,Asian Restaurant,Pub,Japanese Restaurant,Pharmacy,Park,Falafel Restaurant
13,North York,Don Mills,2.0,Gym,Coffee Shop,Asian Restaurant,Clothing Store,Sporting Goods Shop,Beer Store,Restaurant,ATM,Sandwich Place,Supermarket
14,East York,Woodbine Heights,2.0,Skating Rink,ATM,Asian Restaurant,Construction & Landscaping,Cosmetics Shop,Pharmacy,Curling Ice,Dance Studio,Diner,Bus Stop
15,Downtown Toronto,St. James Town,2.0,Café,Coffee Shop,Restaurant,Italian Restaurant,Hotel,Cosmetics Shop,Breakfast Spot,Diner,Seafood Restaurant,Gastropub
16,York,Humewood-Cedarvale,2.0,Business Service,Hockey Arena,Field,Trail,Yoga Studio,Event Space,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant


#### Cluster 3: Handymans paradise, full of 'Construction & Landscaping' venues

In [38]:
cluster_number = 3
cluster = toronto_merged.loc[toronto_merged['Cluster Labels'] == cluster_number, toronto_merged.columns[[1, 2] + list(range(5, toronto_merged.shape[1]))]]
print('cluster has {} postal codes included with color {}'.format(cluster.shape[0], webcolors.hex_to_name(rainbow[cluster_number - 1])))
cluster

cluster has 5 postal codes included with color orange


,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Parkwoods,3.0,Construction & Landscaping,BBQ Joint,Food & Drink Shop,Bus Stop,Fireworks Store,Convenience Store,Park,Coworking Space,Fabric Shop,Eastern European Restaurant
12,Scarborough,Rouge Hill / Port Union / Highland Creek,3.0,Construction & Landscaping,Home Service,Golf Course,Moving Target,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
49,North York,North Park / Maple Leaf Park / Upwood Park,3.0,Construction & Landscaping,Massage Studio,Park,Bakery,Fabric Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant
57,North York,Humberlea / Emery,3.0,Construction & Landscaping,Baseball Field,Paper / Office Supplies Store,Fabric Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space
101,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...,3.0,Construction & Landscaping,Business Service,Home Service,Park,Locksmith,Event Space,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant


#### Cluster 4: for lovers of Nature, full of parks around

In [39]:
cluster_number = 4
cluster = toronto_merged.loc[toronto_merged['Cluster Labels'] == cluster_number, toronto_merged.columns[[1, 2] + list(range(5, toronto_merged.shape[1]))]]
print('cluster has {} postal codes included with color {}'.format(cluster.shape[0], webcolors.hex_to_name(rainbow[cluster_number - 1])))
cluster

cluster has 4 postal codes included with color red


,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,York,Caledonia-Fairbanks,4.0,Park,Women's Store,Spa,Fabric Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant
64,York,Weston,4.0,Park,Convenience Store,Electronics Store,Yoga Studio,Fabric Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,Ethiopian Restaurant
85,Scarborough,Milliken / Agincourt North / Steeles East / L'...,4.0,Park,Yoga Studio,Fabric Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space
91,Downtown Toronto,Rosedale,4.0,Park,Trail,Playground,Yoga Studio,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant


#### Cluster 5: a Residential place, with only a Wendy's around and a Home Service company

In [40]:
cluster_number = 5
cluster = toronto_merged.loc[toronto_merged['Cluster Labels'] == cluster_number, toronto_merged.columns[[1, 2] + list(range(5, toronto_merged.shape[1]))]]
print('cluster has {} postal codes included with color {}'.format(cluster.shape[0], webcolors.hex_to_name(rainbow[cluster_number - 1])))
cluster

cluster has 1 postal codes included with color white


,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,Malvern / Rouge,5.0,Home Service,Fast Food Restaurant,Yoga Studio,Event Space,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant


#### Cluster 6: a Residential place, with only a shop of technology around

In [41]:
cluster_number = 6
cluster = toronto_merged.loc[toronto_merged['Cluster Labels'] == cluster_number, toronto_merged.columns[[1, 2] + list(range(5, toronto_merged.shape[1]))]]
print('cluster has {} postal codes included with color {}'.format(cluster.shape[0], webcolors.hex_to_name(rainbow[cluster_number - 1])))
cluster

cluster has 1 postal codes included with color black


,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Scarborough,Scarborough Village,6.0,Women's Store,Yoga Studio,Fabric Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space


#### Cluster 7: for mads of Karate
This neighboruhood only has three venues: a Park, a Cafeteria and a Karate school

In [42]:
cluster_number = 7
cluster = toronto_merged.loc[toronto_merged['Cluster Labels'] == cluster_number, toronto_merged.columns[[1, 2] + list(range(5, toronto_merged.shape[1]))]]
print('cluster has {} postal codes included with color {}'.format(cluster.shape[0], webcolors.hex_to_name(rainbow[cluster_number - 1])))
cluster

cluster has 1 postal codes included with color gray


,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,North York,York Mills / Silver Hills,7.0,Martial Arts Dojo,Park,Cafeteria,Event Space,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant


#### Cluster 8: neighborhoods with more venues, mainly 'Home Service', but also Pharmacies and Ice Cream shops

In [43]:
cluster_number = 8
cluster = toronto_merged.loc[toronto_merged['Cluster Labels'] == cluster_number, toronto_merged.columns[[1, 2] + list(range(5, toronto_merged.shape[1]))]]
print('cluster has {} postal codes included with color {}'.format(cluster.shape[0], webcolors.hex_to_name(rainbow[cluster_number - 1])))
cluster

cluster has 4 postal codes included with color pink


,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Etobicoke,West Deane Park / Princess Gardens / Martin Gr...,8.0,Home Service,Gift Shop,Print Shop,Yoga Studio,Ethiopian Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
50,North York,Humber Summit,8.0,Pharmacy,Pizza Place,Home Service,Event Space,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant
53,North York,Downsview,8.0,Home Service,Business Service,Korean Restaurant,Baseball Field,Yoga Studio,Fabric Shop,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space
62,Central Toronto,Roselawn,8.0,Home Service,Health & Beauty Service,Ice Cream Shop,Pool,Field,Ethiopian Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant


#### Cluster 9: for people who likes to stay at home... no place to go here
The only venue here is a 'moving company', to help you change to other house

In [44]:
cluster_number = 9
cluster = toronto_merged.loc[toronto_merged['Cluster Labels'] == cluster_number, toronto_merged.columns[[1, 2] + list(range(5, toronto_merged.shape[1]))]]
print('cluster has {} postal codes included with color {}'.format(cluster.shape[0], webcolors.hex_to_name(rainbow[cluster_number - 1])))
cluster

cluster has 1 postal codes included with color cyan


,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
52,North York,Willowdale / Newtonbrook,9.0,Home Service,Yoga Studio,Fabric Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space


#### Cluster 10: the total outlier... with no venue around, fully into the nature
Upper Rouge is located into the 'Rouge National Urban Park', so full nature everywhere, I think...

In [45]:
cluster_number = 10
cluster = toronto_merged.loc[toronto_merged['Cluster Labels'] == cluster_number, toronto_merged.columns[[1, 2] + list(range(5, toronto_merged.shape[1]))]]
print('cluster has {} postal codes included with color {}'.format(cluster.shape[0], webcolors.hex_to_name(rainbow[cluster_number - 1])))
cluster

cluster has 1 postal codes included with color brown


,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
95,Scarborough,Upper Rouge,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 7. Conclusions <a name="conclusions"></a>

* I have run the model with **10 clusters** (as you can see above) **and with 5 clusters**. The result is mostly the same:
    * **1 big cluster around 'Coffe Shops'**
    * **1 mid-size cluster around 'construction & Landscaping'**
    * **1 mid-size cluster around 'Parks'**
    * **The rest are mostly 'outliers'** (clusters of 1/4 neighborhoods): residential neighborhoods with only a few venues, sometimes 'Home Service' venues, sometimes 'fast-food' or 'slow-food' restaurants, sometimes a 'Karate dojo'... but usually less than 3 venues. They could be in the same cluster if 'number of venues' was a classification element also
<BR>**Maybe 4 clusters wuold have been enough.**
* It is very difficult to take decissions only on these elements if you don't know the city: **more information/analysis would be required to take a better decission**

Anyway, a very interesting exercise to do.

Thanks for reviewing!